In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import os

# 爬蟲


def web_crawl(drug_name):

    # CSV 檔案路徑
    csv_file_path = "drug_products.csv"

    # 如果檔案存在，先刪除
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)

    medicines = [
        "Alaxan",
        "Bactidol",
        "Bioflu",
        "Biogesic",
        "DayZinc",
        "Decolgen",
        "Fish Oil",
        'Fishoil',
        "Kremil S",
        "Medicol",
        "Neozep"
    ]

    if drug_name not in medicines:
        return False

    # 圖像辨識出來的 'Fishoil' 查不到資料，但是 'Fish Oil'查得到
    if drug_name == 'Fishoil':
        drug_name = 'Fish Oil'

    print("正在進行爬蟲(chrome)，使用無頭模式所以不會有視窗")

    # 設定下載目錄（根據需求修改）
    download_directory = "/mnt/d/drug_yolo_webcrawling"  # 建議使用 /tmp 或其他有效的 Linux 路徑

    # 確保下載目錄存在
    if not os.path.exists(download_directory):
        os.makedirs(download_directory)

    # 設置 Chrome 選項
    options = Options()

    options.add_argument("--headless")  # 無頭模式
    options.add_argument("--no-sandbox")  # 禁用沙箱
    options.add_argument("--disable-dev-shm-usage")  # 共享內存問題
    options.add_argument("--disable-gpu")  # 禁用 GPU
    options.add_argument("--disable-notifications")  # 禁用通知彈窗
    options.add_argument("--window-size=1920x1080")  # 設置窗口大小，防止某些元素不可見

    print(options.arguments)  # 确认 "--headless" 是否包含在选项列表中

    # 設置Chrome選項
    options = Options()
    options.add_experimental_option(
        "prefs",
        {
            "download.default_directory": download_directory,  # 設定預設下載目錄
            "download.prompt_for_download": False,  # 禁止顯示下載提示框
            "download.directory_upgrade": True,  # 允許覆蓋舊的下載文件
        })

    # 創建webdriver實例
    chrome = webdriver.Chrome(options=options)
    try:
        # 打開目標網站
        # drug_name = "Decolgen"

        url = f"https://verification.fda.gov.ph/drug_productslist.php?cmd=search&t=drug_products&psearch={drug_name}&psearchtype="

        chrome.get(url)

        # 等待網站加載完成
        time.sleep(3)  # 可根據網站載入速度調整

        # 找到並點擊觸發文件下載的按鈕
        export_button = chrome.find_element(By.CSS_SELECTOR,
                                            "button.dropdown-toggle")
        export_button.click()

        # 模擬點擊導出選項（例如選擇 CSV 格式）
        csv_button = chrome.find_element(By.PARTIAL_LINK_TEXT, "CSV")
        csv_button.click()

        # 等待文件下載完成，根據文件大小和網速調整時間
        time.sleep(3)

        print(f"文件已下載到：{download_directory}")

    except Exception as e:
        print(f"執行過程中發生錯誤: {e}")

    finally:
        # 關閉瀏覽器
        chrome.close()

    return True

In [ ]:
# 測試爬蟲

# input_text = "Fishoil"
# web_crawl(input_text)

In [ ]:
import pandas as pd

# 讀取CSV檔案


def read_csv():

    # CSV 檔案路徑
    csv_file_path = "drug_products.csv"

    try:
        # 讀取 CSV，取前 10 行作為預覽
        df = pd.read_csv(csv_file_path)

        # 檢查是否有 "Product Information" 欄位，並刪除它
        if "Product Information" in df.columns:
            df = df.drop(columns=["Product Information"])

        return df  # 返回整個 DataFrame
    except FileNotFoundError:
        return "CSV file not found. Please ensure the file is available."
    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
# 測試讀取CSV檔案

# result = read_csv()
# print(result)

In [ ]:
# YOLOv5
import cv2
import numpy as np
import torch
from pathlib import Path

# yolov5 訓練的權重


def detect_image_yolov5(img):
    # 输入图片路径和权重路径
    # pic_path = 'test_image.jpg'
    weights_path = "best_yolov5.pt"

    # 加载模型
    model = torch.hub.load('ultralytics/yolov5',
                           'custom',
                           path=weights_path,
                           force_reload=True)

    # 加载图片并推理
    # img = cv2.imread(pic_path)
    results = model(img)

    # 获取检测结果
    df = results.pandas().xyxy[0]
    print(df)

    # 提取辨識結果中的 name 欄位並且不重複
    name_list = list(set(df['name'].astype(str)))  # 转换为字符串，以防有不同类型的数据

    # 保存渲染结果
    rendered_img = np.squeeze(results.render()).astype(np.uint8)
    output_path = 'result.png'
    cv2.imwrite(output_path, rendered_img)
    print(f"Result saved as {output_path}")

    return name_list

In [ ]:
# 測試 yolov5

# pic_path = 'no_medicine_image.jpg'
# img = cv2.imread(pic_path)
# name_list = detect_image_yolov5(img)
# print(name_list)

In [ ]:
# YOLOv11
import cv2
import numpy as np
from ultralytics import YOLO
import os


def detect_image_yolov11(img):

    # CSV 檔案路徑
    csv_file_path = "result.png"

    # 如果檔案存在，先刪除
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)

    # 输入模型权重路径
    weights_path = "best_yolov11x.pt"

    # 加载 YOLOv8 模型
    model = YOLO(weights_path)

    # 推理
    results = model.predict(source=img, save=False)

    # 获取检测结果
    detections = results[0]  # YOLOv8 支持批量预测，这里只取第一个结果
    boxes = detections.boxes.xyxy.cpu().numpy()  # 检测框 [x1, y1, x2, y2]
    scores = detections.boxes.conf.cpu().numpy()  # 置信分数
    class_ids = detections.boxes.cls.cpu().numpy()  # 类别编号
    names = model.names  # 类别名称映射表

    # 提取检测的类别名称并去重
    name_list = list(set([names[int(cls_id)] for cls_id in class_ids]))

    # 渲染并保存结果图片
    rendered_img = results[0].plot()  # 使用 YOLOv8 内置的绘图方法
    output_path = 'result.png'
    cv2.imwrite(output_path, rendered_img)
    print(f"Result saved as {output_path}")

    # 打印检测结果
    print(f"Detected names: {name_list}")

    return name_list

In [ ]:
# 測試 yolov11

# pic_path = 'test_images/Fish Oil.jpg'
# img = cv2.imread(pic_path)
# name_list = detect_image_yolov11(img)
# print(name_list)

In [ ]:
import gradio as gr

# 定義藥品查詢功能
medicines = [
    "Alaxan",
    "Bactidol",
    "Bioflu",
    "Biogesic",
    "DayZinc",
    "Decolgen",
    "Fish Oil",
    "Kremil S",
    "Medicol",
    "Neozep"
]


def crawl_by_name(input_text):

    if input_text in medicines:
        web_crawl(input_text)
        return f"{input_text} is in the medicine list."
    else:
        return f"{input_text} is not in the medicine list."


def crawl_by_image(image):
    name_list = detect_image_yolov11(image)

    # 圖像辨識出來的 'Fishoil' 查不到資料，但是 'Fish Oil'查得到
    for i in name_list:
        if i == 'Fishoil':
            i = 'Fish Oil'
            break

    print(name_list)

    # 如果没有检测到药品
    if len(name_list) == 0:
        return "No medicine is detected in this image."

    # 如果检测到多个药品
    elif len(name_list) > 1:
        # 列出所有检测到的药品
        detected_medicines = ", ".join(name_list)
        # 输出消息，说明只会搜索第一个药品
        output_str = f"Medicines detected: {detected_medicines}.\nMultiple medicines are detected in this image.\nWill search for the first medicine detected: {name_list[0]}."
        web_crawl(name_list[0])  # 只搜索第一个药品
        return output_str

    # 如果只检测到一个药品
    else:
        output_str = f"Medicine detected: {name_list[0]}.\nWill search for this medicine."
        web_crawl(name_list[0])  # 搜索这个药品
        return output_str


def load_image():
    return "result.png"


# 創建 Gradio 介面
with gr.Blocks() as demo:
    with gr.Tabs():
        # 第一個標籤：Detect Medicine Image
        with gr.TabItem("Detect Medicine Image"):
            gr.Markdown("## Detect Medicine Image")
            image_input = gr.Image(
                type="pil", label="Input Medicine Image", interactive=True)
            upload_btn = gr.Button("Upload Image")
            result = gr.Textbox(label="Result", lines=3)
            upload_btn.click(crawl_by_image, inputs=image_input,
                             outputs=result)

        # 第二個標籤：Search Medicine Name
        with gr.TabItem("Search Medicine Name"):
            gr.Markdown("## Search Medicine Name")
            with gr.Row():
                text_input = gr.Textbox(
                    label="Input Medicine Name",
                    lines=1,
                    placeholder="Enter medicine name here..."
                )
            process_btn = gr.Button("Submit")
            result = gr.Textbox(label="Result", lines=3)
            process_btn.click(crawl_by_name, inputs=text_input, outputs=result)

        # 第三個標籤：CSV 資料檢視
        with gr.TabItem("View Medicine data"):
            gr.Markdown("## View Medicine data")
            view_csv_btn = gr.Button("Load Medicine data")
            csv_output = gr.Dataframe(
                value=None,  # 初始無內容
                headers=None,  # 默認不顯示標題
                datatype="str",  # 指定數據類型為字串，適用於混合型資料
                interactive=False,  # 設為非互動模式
                max_height=300  # 設定表格最大高度
            )
            view_csv_btn.click(read_csv, inputs=[], outputs=csv_output)

        # 第四個標籤：View Result Image
        with gr.TabItem("View Result Image"):
            gr.Markdown("## View Result Image")
            load_image_btn = gr.Button("Load Image")
            result_image = gr.Image(label="Result Image")
            load_image_btn.click(load_image, inputs=[], outputs=result_image)

In [ ]:
# 關閉服务
demo.close()

In [ ]:
# 啟動介面
demo.launch()